# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.09,77,100,21.18,TF,1718063590
1,1,port blair,11.6667,92.7500,28.02,89,75,6.17,IN,1718063590
2,2,nambucca heads,-30.6500,153.0000,17.00,54,30,1.17,AU,1718063590
3,3,pangai,-19.8000,-174.3500,27.06,57,40,5.66,TO,1718063590
4,4,aldinga beach,-35.2793,138.4604,17.07,87,98,8.50,AU,1718063590


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [18]:
%%capture --no-display
import warnings
warnings.filterwarnings("ignore")

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Narrow down cities that fit criteria and drop any results with null values
temp = city_data_df.loc[city_data_df["Max Temp"] >= 21]
temp_2 = temp.loc[temp["Max Temp"] <= 27]
temp_2.head()

wind = temp_2.loc[temp_2["Wind Speed"] <= 4.5]
cloud = wind.loc[wind['Cloudiness'] == 0]
cloud.head()
# Drop any rows with null values
cloud.dropna()

# Display sample data
cloud

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,aral,40.5156,81.2636,26.06,26,0,2.12,CN,1718063590
70,70,chaman,30.9236,66.4512,24.52,9,0,2.01,PK,1718063593
252,252,markaz-e hukumat-e darweshan,31.1323,64.1934,26.38,12,0,0.91,AF,1718063601
290,290,carroll,39.5501,-77.0164,23.18,56,0,1.54,US,1718063603
314,314,al jawf,29.5000,38.7500,24.21,30,0,2.22,SA,1718063604
382,382,tobruk,32.0836,23.9764,23.34,72,0,2.73,LY,1718063607
386,386,bet dagan,32.0014,34.8303,25.83,90,0,3.09,IL,1718063608
411,411,san luis del palmar,-27.5079,-58.5545,24.44,73,0,2.69,AR,1718063609
433,433,suez,29.9737,32.5263,23.22,77,0,3.82,EG,1718063610
467,467,north ward,-19.2454,146.8075,22.95,60,0,2.06,AU,1718063612


### Step 3: Create a new DataFrame called `hotel_df`.

In [20]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cloud.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

hotel_df.head()

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,aral,40.5156,81.2636,26.06,26,0,2.12,CN,1718063590,
70,70,chaman,30.9236,66.4512,24.52,9,0,2.01,PK,1718063593,
252,252,markaz-e hukumat-e darweshan,31.1323,64.1934,26.38,12,0,0.91,AF,1718063601,
290,290,carroll,39.5501,-77.0164,23.18,56,0,1.54,US,1718063603,
314,314,al jawf,29.5000,38.7500,24.21,30,0,2.22,SA,1718063604,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aral - nearest hotel: No hotel found
chaman - nearest hotel: No hotel found
markaz-e hukumat-e darweshan - nearest hotel: No hotel found
carroll - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
tobruk - nearest hotel: No hotel found
bet dagan - nearest hotel: No hotel found
san luis del palmar - nearest hotel: No hotel found
suez - nearest hotel: No hotel found
north ward - nearest hotel: No hotel found
vila velha - nearest hotel: No hotel found
lenino - nearest hotel: No hotel found
alamosa - nearest hotel: No hotel found
gunjur - nearest hotel: No hotel found
center - nearest hotel: No hotel found
khamis mushait - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,7,aral,40.5156,81.2636,26.06,26,0,2.12,CN,1718063590,No hotel found
70,70,chaman,30.9236,66.4512,24.52,9,0,2.01,PK,1718063593,No hotel found
252,252,markaz-e hukumat-e darweshan,31.1323,64.1934,26.38,12,0,0.91,AF,1718063601,No hotel found
290,290,carroll,39.5501,-77.0164,23.18,56,0,1.54,US,1718063603,No hotel found
314,314,al jawf,29.5000,38.7500,24.21,30,0,2.22,SA,1718063604,No hotel found
382,382,tobruk,32.0836,23.9764,23.34,72,0,2.73,LY,1718063607,No hotel found
386,386,bet dagan,32.0014,34.8303,25.83,90,0,3.09,IL,1718063608,No hotel found
411,411,san luis del palmar,-27.5079,-58.5545,24.44,73,0,2.69,AR,1718063609,No hotel found
433,433,suez,29.9737,32.5263,23.22,77,0,3.82,EG,1718063610,No hotel found
467,467,north ward,-19.2454,146.8075,22.95,60,0,2.06,AU,1718063612,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
import warnings
warnings.filterwarnings("ignore")

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]